In [5]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('vn_stock_companies.csv')

# Filter the rows where group_code is 'VNINDEX' or 'HNX'
vnindex_df = df[df['group_code'] != 'UpcomIndex']

# Get the list of tickers
symbollist = vnindex_df['ticker'].tolist()
# Lấy mã cổ phiếu không lấy mã cổ phiếu quỹ
filtered_symbollist = [symbol for symbol in symbollist if len(symbol) <= 4]

In [7]:
from vnstock import *
import numpy as np
financial_data = []  # List to store the financial data objects

for symbol in filtered_symbollist:
    try:
        data = financial_report(symbol=symbol, report_type='IncomeStatement', frequency='Yearly')
        
        # Remove the 'Q5 ' from years
        data.columns = [col.replace('Q5 ', '') for col in data.columns]
        
        # Find the row index where "CHỈ TIÊU" matches one of the three values
        row_index = data[data['CHỈ TIÊU'].str.contains('Lợi nhuận của Cổ đông của Công ty mẹ|Lợi nhuận sau thuế của chủ sở hữu, tập đoàn|Lợi nhuận sau thuế phân bổ cho chủ sở hữu|Lợi nhuận sau thuế')].index[0]
        
        # Use the row index to get the net income data
        net_income = data.loc[row_index].to_dict()
        
        # Remove the first entry in the dictionary
        del net_income['CHỈ TIÊU']
        
        years = list(net_income.keys())[5:]
        compound_rate = {}
        
        for year in years:
            year_int = int(year)
            sum_net_income_5_years = sum(net_income[str(y)] for y in range(year_int - 5, year_int))
            compound_rate[year] = (net_income[year] - net_income[str(year_int - 5)]) / sum_net_income_5_years
            
        # Calculate average 5 year ROE
        df = financial_ratio(symbol, 'yearly', True)
        final_average_roe = 0.0
        if 'roe' in df.columns and len(df) >= 5:
            average_roe = df['roe'].head(5).mean()
            if not np.isnan(average_roe):
                final_average_roe = average_roe
        
        financial_data.append({'ticker': symbol, 'net_income': net_income, 'compound_rate': compound_rate, 'average_5y_roe': final_average_roe})
    except Exception as e:
        print(f"Error fetching data for symbol {symbol}: {str(e)}")
        continue


Error fetching data for symbol VTZ: Excel file format cannot be determined, you must specify an engine manually.
Error fetching data for symbol GMH: Excel file format cannot be determined, you must specify an engine manually.
Error fetching data for symbol HMR: Excel file format cannot be determined, you must specify an engine manually.
Error fetching data for symbol PCH: Excel file format cannot be determined, you must specify an engine manually.
Error fetching data for symbol NO1: Excel file format cannot be determined, you must specify an engine manually.
Error fetching data for symbol ACB: invalid literal for int() with base 10: 'Q5 2018'
Error fetching data for symbol CAG: Excel file format cannot be determined, you must specify an engine manually.
Error fetching data for symbol AGG: Excel file format cannot be determined, you must specify an engine manually.
Error fetching data for symbol ACG: Excel file format cannot be determined, you must specify an engine manually.
Error fetc

In [8]:
for entry in financial_data:
    compound_rates = entry['compound_rate']
    latest_years = list(compound_rates.keys())[-5:]
    average_5y_compound_rate = sum(compound_rates[year] for year in latest_years) / 5
    entry['average_5y_compound_rate'] = average_5y_compound_rate

# sort financial_data based on the highest roe and average_5y_compound_rate
financial_data = sorted(financial_data, key=lambda x: (x['average_5y_roe'], x['average_5y_compound_rate']), reverse=True)


In [9]:
import json

# ... your code to populate financial_data ...

# Define the file path to save the data
file_path = 'financial_data.json'

# Write financial_data to the file in JSON format
with open(file_path, 'w') as file:
    json.dump(financial_data, file)

In [12]:
data = financial_report(symbol="ACB", report_type='IncomeStatement', frequency='Yearly')
data.columns = [col.replace('Q5 ', '') for col in data.columns]
data

,CHỈ TIÊU,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Thu nhập lãi và các khoản thu nhập tương tự,1.538447e+13,1.370283e+13,1.408179e+13,1.644825e+13,2.031964e+13,2.401536e+13,2.831752e+13,3.185575e+13,3.371381e+13,4.069883e+13
1,Chi phí lãi và các chi phí tương tự,-1.081866e+13,-8.937199e+12,-8.198265e+12,-9.556360e+12,-1.186188e+13,-1.365244e+13,-1.620536e+13,-1.727362e+13,-1.476902e+13,-1.716530e+13
2,Thu nhập lãi thuần,4.565813e+12,4.765633e+12,5.883527e+12,6.891889e+12,8.457754e+12,1.036292e+13,1.211216e+13,1.458212e+13,1.894479e+13,2.353353e+13
3,Thu nhập từ hoạt động dịch vụ,8.172430e+11,9.446820e+11,1.020989e+12,1.274131e+12,1.574668e+12,2.094599e+12,2.708759e+12,2.803484e+12,4.026460e+12,5.057559e+12
4,chi phí hoạt động dịch vụ,-2.262230e+11,-2.502420e+11,-2.757630e+11,-3.297490e+11,-3.863370e+11,-5.970730e+11,-8.122670e+11,-1.108830e+12,-1.132497e+12,-1.531315e+12
5,Lãi thuần từ hoạt động dịch vụ,5.910200e+11,6.944400e+11,7.452260e+11,9.443820e+11,1.188331e+12,1.497526e+12,1.896492e+12,1.694654e+12,2.893963e+12,3.526244e+12
6,Lãi/(lỗ) thuần từ hoạy động kinh doanh ngoại h...,-7.761600e+10,1.836340e+11,1.206240e+11,2.300960e+11,2.367290e+11,2.413900e+11,4.303250e+11,6.871870e+11,8.715560e+11,1.048369e+12
7,Lãi/(lỗ) thuần từ mua bán chứng khoán kinh doanh,6.999200e+10,1.103730e+11,1.454400e+10,7.208300e+10,2.530500e+10,-7.832900e+10,7.520600e+10,1.665030e+11,4.499600e+11,-3.878730e+11
8,Lãi/(lỗ) thuần từ mua bán chứng khoán đầu tư,3.963950e+11,1.326720e+11,-8.076000e+11,-8.859630e+11,6.030790e+11,1.685340e+11,5.432200e+10,7.321150e+11,2.440510e+11,2.064800e+10
9,Thu nhập từ hoạt động khác,7.673900e+10,6.703200e+10,2.755110e+11,2.962850e+11,9.524390e+11,1.952302e+12,1.546194e+12,4.954590e+11,7.031590e+11,1.110494e+12
